In [3]:
import pickle
import pandas as pd
import re

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report 
from sklearn.decomposition import NMF,LatentDirichletAllocation
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
us = pickle.load(open("wikivoyage_text_US.p","rb"))

In [6]:
df = pd.DataFrame.from_dict(us,orient='index')

In [7]:
df = df.reset_index()

In [8]:
df = df.rename(columns=({ 'index' : 'Name'}))

In [9]:
df.head()

,Name,text,type,loc
0,AbercrombiePAGE,{{pagebanner|Abercrombie WikiVoyage Banner ND....,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
1,Aberdeen (Maryland)PAGE,{{pagebanner|Aberdeen MD WikiVoyage Banner.jpg...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
2,Aberdeen (South Dakota)PAGE,{{pagebanner|Pagebanner default.jpg|pgname=Abe...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
3,Aberdeen (Washington)PAGE,{{Pagebanner|pgname=Aberdeen |Wikivoyage page ...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
4,AbernathyPAGE,{{pagebanner|Abernathy Texas Wikivoyage Banner...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...


In [10]:
def unwiki(wiki):
        """
       Remove wiki markup from the text.
       """
        wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
        wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
        wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
        wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r"''+", '', wiki)
        wiki = re.sub(r'(?m)^\*$', '', wiki)
       
        return wiki

In [11]:
def unhtml(html):
        """
       Remove HTML from the text.
       """
        html = re.sub(r'(?i)&nbsp;', ' ', html)
        html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
        html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
        html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
        html = re.sub(r'(?m)<.*?>', '', html)
        html = re.sub(r'(?i)&amp;', '&', html)
       
        return html

In [12]:
def f(x):
    return unhtml(unwiki(x))

In [13]:
df['text'] = df['text'].apply(f)

In [ ]:
vect = CountVectorizer(stop_words="english", max_features = 5000)

In [12]:
features = df['text']

In [ ]:
x_features = vect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = vect.get_feature_names()

In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic #%d:" % topic_idx
        print " ".join(sorted([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]],key=lambda x: len(x),reverse=True))

In [ ]:
print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print df['Name'][max_2t]

In [ ]:
list(df[df['Name']=='Touring prestigious and notable universities in the U.S.PAGE']['loc'])

In [14]:
pages = df[df['type'] == 'page']

In [15]:
def f(x):
    return re.sub('PAGE', '', x)

In [16]:
pages['Name'] = pages['Name'].apply(f)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
len("//tools.wmflabs.org/wikivoyage/w/poimap2")

In [ ]:
pages.head()

In [17]:
pages = pages[pages['loc'].str.contains("//tools.wmflabs.org/wikivoyage/w/poimap2")==True]

In [18]:
from nltk.corpus import stopwords

In [19]:
StopWords = stopwords.words('english')

In [20]:
StopPlaces = list(pages['Name'])

In [21]:
StopPlaces = map(lambda x: re.sub("\(|\)", '',x), StopPlaces)

In [22]:
StopPlaces = map(lambda x: x.lower().split(), StopPlaces)

In [23]:
StopPlaces = [item for sublist in StopPlaces for item in sublist]

In [24]:
StopWords = StopPlaces + StopWords

In [25]:
StopWords = list(set(StopWords))

In [ ]:
vect = CountVectorizer(stop_words=StopWords)

In [18]:
features = pages['text'].apply(lambda x: x.lower())

In [ ]:
x_features = vect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = vect.get_feature_names()

In [ ]:
print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages['Name'])[max_2t]

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [28]:
### TF-IDF Vectorizer

In [29]:
StopWords = StopWords + map(lambda x: x.split("/"), StopWords)

In [30]:
StopWords = [item for sublist in StopWords for item in sublist]

In [30]:
TFIDFvect = TfidfVectorizer(stop_words=StopWords)

In [31]:
x_features = TFIDFvect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = TFIDFvect.get_feature_names()

In [ ]:
print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages['Name'])[max_2t]

### LDA

In [32]:
lda = LatentDirichletAllocation(n_topics=20, random_state=1)
W = lda.fit_transform(x_features)
H = lda.components_

In [33]:
print_top_words(lda,vector_features, 50)

NameError: name 'vector_features' is not defined

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages['Name'])[max_2t]

In [ ]:
pages[pages['Name'] == 'Buffalo/East Side']

In [ ]:
print len(pages['text'].ix[801])

In [19]:
def f(x):
    x = re.sub(r'WikiPedia:([\s\S]*)','', x)
    x = re.sub(r'Dmoz:([\s\S]*)','', x)
    x = re.sub(r'\[([\s\S]*)\]','', x)
    x = re.sub(r'[0-9]','', x)
    x = re.sub(r'\=\=Get in\=\=([\s\S]*)(?=\=\=Get around\=\=)','',x)
    x = re.sub(r'\=\=Get around\=\=([\s\S]*)(?=\=\=See\=\=)','',x)
    x = re.sub(r'\=\=Respect\=\=([\s\S]*)(?=\=\=Go next\=\=)','',x)
    x = re.sub(r'\=\=Stay safe\=\=([\s\S]*)(?=\=\=Go next\=\=)','',x)
    x = re.sub(r'\=\=([\S ]*)\=\=', '', x)
    x = re.sub(r'\=\=Go next\=\=([\s\S]*)','', x)
    return x

In [20]:
pages['regex_text'] = pages['text'].apply(f)

In [21]:
pages['len'] = map(lambda x: len(x), pages['regex_text'])

In [22]:
pages2 = pages[pages['len']> 1000]

In [23]:
len(pages2)

2516

In [40]:
from gensim.models.doc2vec import LabeledSentence

In [130]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc,tags=[self.labels_list[idx]])

In [194]:
np.array([i for i,j in enumerate(list(pages2["Name"])) if "Airport" not in j])

array([   0,    1,    2, ..., 2513, 2514, 2515])

In [195]:
pages3 = pages2.reset_index()

In [198]:
pages3 = pages3.ix[np.array([i for i,j in enumerate(list(pages2["Name"])) if "Airport" not in j])]

In [201]:
features = list(pages3['regex_text'].apply(lambda x: "".join(x.lower().splitlines())))

In [202]:
features = [tokenizer.tokenize(x) for x in features]

In [203]:
labels_list = list(pages3['Name'])

In [204]:
from nltk.tokenize import RegexpTokenizer

In [205]:
tokenizer = RegexpTokenizer(r'\w+')

In [206]:
it = LabeledLineSentence(features, labels_list)

In [128]:
import gensim

In [207]:
model = gensim.models.Doc2Vec(size=300, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate

In [208]:
model.build_vocab(it)

In [209]:
for epoch in range(10):
    model.train(it)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(it)

In [210]:
model.save("doc2vec.model")

In [239]:
model.docvecs.most_similar(positive=["Las Vegas", "Texas"],negative=["Nevada"])

[(u'Texas Panhandle', 0.25897830724716187),
 (u'Lubbock', 0.23987515270709991),
 (u'Garland (Texas)', 0.23293527960777283),
 (u'Montgomery', 0.22636441886425018),
 (u'Baltimore/Downtown', 0.22568699717521667),
 (u'Irving', 0.21994465589523315),
 (u'Walt Disney World/Disney Springs', 0.21741271018981934),
 (u'Hoboken', 0.2170657366514206),
 (u'South Texas Plains', 0.2145078182220459),
 (u'Albuquerque', 0.20683950185775757)]

In [139]:
model[u'Aberdeen (Washington)']

AttributeError: 'Doc2Vec' object has no attribute 'syn0'

In [174]:
pages2[pages2['Name'] == "New York (state)"]['Name']

4787    New York (state)
Name: Name, dtype: object

In [37]:
TFIDFvect = TfidfVectorizer(stop_words=StopWords,max_features=5000)
x_features = TFIDFvect.fit_transform(features)

In [38]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [39]:
vector_features = TFIDFvect.get_feature_names()

print_top_words(nmf, vector_features, 50)

Topic #0:
although however looking usually people around places locals though really pretty hotels always visit night local check would quite right often small good find also like want even much time take best food many make well bars nice sure less look away get one see lot try re go ll
Topic #1:
neighborhoods neighborhood residential commercial industrial buildings streetcar broadway stopping building streets african century section running parking housing walking avenue routes blocks subway trains though around campus public claude along lines buses stops crime still broad urban line runs stop walk ave bus art st th rd nd av ne dc
Topic #2:
temperatures reservations backcountry campgrounds temperature conditions elevations campground formations available elevation visitors entrance visitor species camping weather average allowed climate permits animals degrees located canyons usually lodging summer within permit trails months inches ranger common season areas found years winds trees

In [40]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages2['Name'])[max_2t]

TOPIC: # 0
Walt Disney World
Saint Thomas
New Orleans
Great Falls (Montana)
Walt Disney World/Magic Kingdom
Somers (Connecticut)
Santa Fe (New Mexico)
Baltimore/Inner Harbor
Blowing Rock
Rochester (New York)
TOPIC: # 1
Manhattan/Upper West Side
Manhattan/Upper East Side
Manhattan/Chelsea
Manhattan/Midtown East
Washington, D.C./Shaw
Manhattan/Theater District
Manhattan/Gramercy Flatiron
Manhattan/Harlem and Upper Manhattan
Manhattan/East Village
Minneapolis/Southeast
TOPIC: # 2
Great Smoky Mountains National Park
Rocky Mountain National Park
Arches National Park
Mount Rainier National Park
White Sands National Monument
Chiricahua National Monument
Yosemite National Park
Timpanogos Cave National Monument
Sequoia and Kings Canyon National Parks
Yellowstone National Park
TOPIC: # 3
Bangor (Pennsylvania)
Nazareth (Pennsylvania)
Seekonk
Eastham
Brewster (Massachusetts)
Ashland (Ohio)
Sandwich (Massachusetts)
Sturbridge
Hackettstown
Kapolei
TOPIC: # 4
Tuolumne County
Albia
Mayfield (Utah)
Emi

In [41]:
from textblob import TextBlob

In [42]:
pages2['tb'] = pages2['text'].map(lambda x: TextBlob(x))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
pages2['polarity'] = pages2['tb'].map(lambda x: x.sentiment[0])
pages2['subjectivity'] = pages2['tb'].map(lambda x: x.sentiment[1])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [44]:
pages2['lat'] = pages2['loc'].map(lambda x: re.findall(r'lat\=([0-9.]*)(?=\&)',x)[0])

IndexError: list index out of range

In [45]:
pages2['lon'] = pages2['loc'].map(lambda x: re.findall(r'lon\=([\-0-9.]*)(?=\&)',x)[0])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [46]:
from collections import Counter

In [91]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
cluster_labels = clusterer.fit_predict(x_features)

In [92]:
Counter(cluster_labels)

Counter({-1: 2288,
         0: 13,
         1: 11,
         2: 9,
         3: 13,
         4: 9,
         5: 26,
         6: 27,
         7: 32,
         8: 21,
         9: 6,
         10: 10,
         11: 9,
         12: 33,
         13: 9})

In [72]:
pages3 = pages2.reset_index()

In [99]:
pages3.ix[np.where(cluster_labels == 13)[0]]

,index,Name,text,type,loc,regex_text,len,tb,polarity,subjectivity,lon
29,92,Alta (Utah),\nAlta is a small town at the head of Little C...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\nAlta is a small town at the head of Little C...,2739,"(\n, A, l, t, a, , i, s, , a, , s, m, a, l,...",0.110258,0.431247,-111.637222
158,429,Beaver Creek,\nBeaver Creek is a ski resort in Colorado.\n\...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\nBeaver Creek is a ski resort in Colorado.\n\...,1921,"(\n, B, e, a, v, e, r, , C, r, e, e, k, , i,...",0.191499,0.510305,-106.521667
269,695,Breckenridge (Colorado),\n\nBreckenridge is a ski resort and town at t...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\n\nBreckenridge is a ski resort and town at t...,1990,"(\n, \n, B, r, e, c, k, e, n, r, i, d, g, e, ...",0.145347,0.376436,-106.03833
876,3039,Government Camp,\nGovernment Camp is a city in Oregon.\n\n==Un...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\nGovernment Camp is a city in Oregon.\n\n\nGo...,2126,"(\n, G, o, v, e, r, n, m, e, n, t, , C, a, m,...",0.147991,0.288876,-121.756667
1140,3712,Keystone (Colorado),\nKeystone is a ski resort in Colorado.\n\n==G...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\nKeystone is a ski resort in Colorado.\n\n\nW...,2134,"(\n, K, e, y, s, t, o, n, e, , i, s, , a, ,...",0.249995,0.484613,-105.948056
1178,3831,Lake Tahoe,\n\n\nLake Tahoe is a popular vacation and rec...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\n\n\nLake Tahoe is a popular vacation and rec...,8470,"(\n, \n, \n, L, a, k, e, , T, a, h, o, e, , ...",0.149287,0.444646,-120.041667
1302,4163,Mammoth Lakes,\n\nMammoth Lakes is a resort city next to Mam...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\n\nMammoth Lakes is a resort city next to Mam...,1986,"(\n, \n, M, a, m, m, o, t, h, , L, a, k, e, s...",0.163889,0.463593,-118.971944
2207,6333,Steamboat Springs,\nSteamboat Springs is a ski resort in the sta...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\nSteamboat Springs is a ski resort in the sta...,4705,"(\n, S, t, e, a, m, b, o, a, t, , S, p, r, i,...",0.128398,0.432170,-106.826667
2327,6677,Vail,\n\n\nVail is a U.S. ski resort town set in th...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...,\n\n\nVail is a U.S. ski resort town set in th...,9618,"(\n, \n, \n, V, a, i, l, , i, s, , a, , U, ...",0.208165,0.490211,-106.363056


In [107]:
vect = CountVectorizer(stop_words=StopWords)
x_features = vect.fit_transform(features)

In [112]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
cluster_labels = clusterer.fit_predict(x_features)

In [113]:
Counter(cluster_labels)

Counter({-1: 1870, 0: 635, 1: 11})

## TODO

In [ ]:
## Consider breaking text into sections (e.g. DO, EAT, etc.)

In [ ]:
## Remove get in/ get around sections

In [ ]:
## Try POS tagging

In [ ]:
## Consider NER for names of restaurants, hotels, etc. so 

In [ ]:
## Consider removing airports
## Combine city sections into one
## Remove National parks and history pages

In [ ]:
## Add European Data